In [1]:
import cv2
import json
import math
import os
import os.path
import re
import tempfile
from matplotlib import pyplot
from util import quickstart as util
from trainer import modelDesign as training_model
from trainer import utils_KITTI as utilkitti

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/jjwong0915/Documents/drone/depth_prediction/.venv/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/jjwong0915/Documents/drone/depth_prediction/.venv/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/jjwong0915/Documents/drone/depth_prediction/.venv/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/jjwong0915/Documents/drone/depth_prediction/.venv/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/home/jjwong0915/Documents/drone/depth_prediction/.venv/lib/python3.7/imp.py", line 342, in l

## Parameters Setting

In [ ]:
# 欲載入 pretrain weight 的路徑
MODEL_WEIGHT_PATH = '/home/islab/Documents/drone/depth_prediction/model/airsim_2020-01-14-16-56_epoch-050_loss-7.1132.h5'

# 資料路徑
DATA_PATH = '/home/islab/Documents/drone/depth_prediction/data/random/2019-12-12-04-54/image'
RESULT_PATH = '/home/islab/Documents/drone/depth_prediction/result/trypfm/2019-12-05-05-49'

# Model Input 解析度設定
INPUT_HEIGHT = 320
INPUT_WIDTH = 384

# Video Output 解析度設定
OUTPUT_HEIGHT = 1440
OUTPUT_WIDTH = 1920

## Build Model

In [ ]:
# 建立 model layer 架構
model = training_model.MNv2_segment_depth_multiloss_model(
    inputShape = (INPUT_HEIGHT, INPUT_WIDTH, 3), 
    alpha = 0.35, 
    expansion_factor = 6, 
    depth_multiplier = 1, 
    lock_backend_weights = False,
    CLASSES = 6,
)

# 載入 model weight
model.load_weights(MODEL_WEIGHT_PATH)

model.summary()

Instructions for updating:
Colocations handled automatically by placer.


/home/islab/.local/share/virtualenvs/depth_prediction-uX_uZt9E/lib/python3.7/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 384, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 [(None, 10, 12, 112) 261728      input_1[0][0]                    
__________________________________________________________________________________________________
subpixel_1 (Lambda)             (None, 20, 24, 28)   0           model_1[1][0]                    
__________________________________________________________________________________________________
conv_expand_1 (Conv2D)          (None, 20, 24, 168)  4704        subpixel_1[0][0]                 
____________________________________________________________________________________________

## Predict the Image and Save Result

In [ ]:
# iterate each dataset
for folder_name in os.listdir(DATA_PATH):
    # prepare result image folder
    result_image_folder_path = os.path.join(RESULT_PATH, 'image', folder_name)
    if not os.path.exists(result_image_folder_path):
        os.makedirs(result_image_folder_path)
    if not os.path.exists(result_image_folder_path.replace('trypfm', 'grey')):
        os.makedirs(result_image_folder_path.replace('trypfm', 'grey'))
    # iterate each image
    folder_path = os.path.join(DATA_PATH, folder_name) 
    for file_name in os.listdir(folder_path):
        # 讀取影像
        scene = util.read_image(os.path.join(folder_path, file_name), shape=(INPUT_HEIGHT,INPUT_WIDTH), rand=0)
        segment = util.read_image(
            os.path.join(folder_path.replace('image', 'seg'), file_name), 
            shape = (INPUT_HEIGHT,INPUT_WIDTH), 
        )
        # 使用 image 來預測出 depth map 和 semantic segmentation
        predict_depth, predict_segment, predict_risk_index, predict_depth_pfm = util.getResult(model, scene, (INPUT_HEIGHT, INPUT_WIDTH))
        #print(predict_risk_index)
        # 儲存結果
        image_path = os.path.join(RESULT_PATH, 'image', folder_name, file_name)
        util.saveResult([scene, segment, predict_depth[0]], predict_risk_index, image_path, (OUTPUT_WIDTH/300, OUTPUT_HEIGHT/300))
        util.save_pfm(image_path.replace('.png', '.pfm'), predict_depth_pfm)    print('predicted dataset {}'.format(folder_name))
            

predicted dataset 2019-12-12-06-49-19
predicted dataset 2019-12-12-09-53-48
predicted dataset 2019-12-12-06-36-26
predicted dataset 2019-12-12-09-00-18
predicted dataset 2019-12-12-05-07-23
predicted dataset 2019-12-12-09-27-23
predicted dataset 2019-12-12-05-52-20
predicted dataset 2019-12-12-10-28-04
predicted dataset 2019-12-12-10-52-38
predicted dataset 2019-12-12-09-08-35
predicted dataset 2019-12-12-06-26-09
predicted dataset 2019-12-12-05-36-14
predicted dataset 2019-12-12-09-13-14
predicted dataset 2019-12-12-05-26-16
predicted dataset 2019-12-12-09-46-31
predicted dataset 2019-12-12-09-26-42
predicted dataset 2019-12-12-05-08-31
predicted dataset 2019-12-12-06-29-48
predicted dataset 2019-12-12-10-42-43
predicted dataset 2019-12-12-10-40-06
predicted dataset 2019-12-12-09-40-05


# Convert Images into Video 

In [4]:
# prepare result video folder
result_video_folder_path = os.path.join(RESULT_PATH, 'video')
if not os.path.exists(result_video_folder_path):
    os.makedirs(result_video_folder_path)
# iterate result images
result_image_folder =  os.path.join(RESULT_PATH, 'image')
for folder_name in os.listdir(result_image_folder):
    # create result video
    video_path = os.path.join(result_video_folder_path, folder_name+'.mp4')
    video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'X264'), 10, (OUTPUT_WIDTH, OUTPUT_HEIGHT))
    # iterate each image
    for file_name in sorted(os.listdir(os.path.join(result_image_folder, folder_name)), key=lambda x: int(re.split('\\-|\\.', x)[-2])):
        # 儲存結果
        image_path = os.path.join(result_image_folder, folder_name, file_name)
        image = cv2.resize(cv2.imread(image_path), (OUTPUT_WIDTH, OUTPUT_HEIGHT))
        video.write(image)
    video.release()
    print('{} created'.format(video_path))

/home/islab/Documents/drone/depth_prediction/result/2019-12-05-05-49/video/2019-12-12-06-49-19.mp4 created
/home/islab/Documents/drone/depth_prediction/result/2019-12-05-05-49/video/2019-12-12-09-53-48.mp4 created
/home/islab/Documents/drone/depth_prediction/result/2019-12-05-05-49/video/2019-12-12-06-36-26.mp4 created
/home/islab/Documents/drone/depth_prediction/result/2019-12-05-05-49/video/2019-12-12-09-00-18.mp4 created
/home/islab/Documents/drone/depth_prediction/result/2019-12-05-05-49/video/2019-12-12-05-07-23.mp4 created
/home/islab/Documents/drone/depth_prediction/result/2019-12-05-05-49/video/2019-12-12-09-27-23.mp4 created
/home/islab/Documents/drone/depth_prediction/result/2019-12-05-05-49/video/2019-12-12-05-52-20.mp4 created
/home/islab/Documents/drone/depth_prediction/result/2019-12-05-05-49/video/2019-12-12-10-28-04.mp4 created
/home/islab/Documents/drone/depth_prediction/result/2019-12-05-05-49/video/2019-12-12-10-52-38.mp4 created
/home/islab/Documents/drone/depth_pre

KeyboardInterrupt: 